In [12]:
import numpy as np
import pandas as pd
import os
import pickle

In [3]:
mypath = os.pardir

In [85]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'District of Columbia': 'DC',
    'Virgin Islands' : 'VI',
    'Puerto Rico' : 'PR',
    'Guam': 'GU'
}

attr_us_state = nv_map = {v: k for k, v in us_state_abbrev.items()}

In [9]:
files = [f for f in os.listdir(mypath) if '.pkl' in f]

In [38]:
weapons_labels = pd.read_excel('Weapon_List.xlsx')

In [39]:
weapons = weapons_labels[weapons_labels['cat'] == 1]
accessories = weapons_labels[weapons_labels['cat'] == 2]
vehicles = weapons_labels[weapons_labels['cat'] == 3]
del weapons_labels

In [40]:
weapons = set(weapons['Item Name'])
accessories = set(accessories['Item Name'])
vehicles = set(vehicles['Item Name'])

In [50]:
all_states_purchases = []
for pkl_file in files:
    pkl_df = pd.read_pickle(mypath+'/'+pkl_file)
    weapons_cost = 0
    accessories_cost = 0
    vehicles_cost = 0
    for _ , row in pkl_df.iterrows():
        if row['Item Name'] in weapons:
            weapons_cost += row['Acquisition Value']
        if row['Item Name'] in accessories:
            accessories_cost += row['Acquisition Value'] 
        if row['Item Name'] in vehicles:
            vehicles_cost += row['Acquisition Value']
    all_states_purchases.append([pkl_file[:-4], weapons_cost, accessories_cost, vehicles_cost])

In [51]:
states_purchases_df = pd.DataFrame(all_states_purchases, columns = ['State', 'Weapons Purchase Amount', 'Tactical Equipment Purchase Amount', 'Vehicle Purchase Amount'])


In [82]:
states_purchases_df.head()

,State,Weapons Purchase Amount,Tactical Equipment Purchase Amount,Vehicle Purchase Amount
0,Missourri,426816.25,1221055.70,2.567322e+07
1,New Mexico,9417.42,1465109.17,2.351108e+07
2,Massachusetts,280203.75,300937.80,7.226124e+06
3,Kentucky,873247.06,791622.36,3.381727e+07
4,California,2940742.92,10648080.72,1.012467e+08


In [88]:
writer = pd.ExcelWriter('State Purchases.xlsx')
states_purchases_df.to_excel(writer,'Sheet1', index = False)
writer.save()

In [93]:
states_purchases_df = pd.read_excel('State Purchases.xlsx')
states_purchases_df.head()

,State,Weapons Purchase Amount,Tactical Equipment Purchase Amount,Vehicle Purchase Amount
0,Missouri,426816.25,1221055.70,2.567322e+07
1,New Mexico,9417.42,1465109.17,2.351108e+07
2,Massachusetts,280203.75,300937.80,7.226124e+06
3,Kentucky,873247.06,791622.36,3.381727e+07
4,California,2940742.92,10648080.72,1.012467e+08


# Importing Census Data

In [102]:
states_census_df = pd.read_excel('All_States_Census_Data.xlsx')

In [103]:
state_list = []
for _, row in states_census_df.iterrows():
    state_list.append(attr_us_state[row['State']])

In [104]:
states_census_df['State'] = state_list

In [105]:
states_census_df.head()

,State,Population,Population Density,HS Completion Rate (%),Higher Education (Bachlors or Higher) (%),Median Household Income ($),Per Capita Income ($),Poverty Rate (%),Black (%),America Indian (%),Asian (%),Native Hawaiian (%),Two or More Races(%),Hispanic (%),White (%)
0,Alabama,4874747,94.4,84.8,24.0,44758,24736,17.1,26.8,0.7,1.4,0.1,1.6,4.2,65.8
1,Alaska,739795,1.2,92.3,28.8,74444,34191,9.9,3.8,15.2,6.3,1.3,7.3,7.0,61.2
2,Arizona,7016270,56.3,86.2,28.0,51340,26686,16.4,4.9,5.4,3.4,0.3,2.8,30.9,55.5
3,Arkansas,3004279,56.0,85.2,21.5,42336,23401,17.2,15.7,1.0,1.6,0.3,2.0,7.3,72.9
4,California,39536653,239.1,82.1,32.0,63783,31458,14.3,6.5,1.7,14.8,0.5,3.8,38.9,37.7


# Combining Census Data to Purchase Data

In [111]:
state_weapon_cost = []
state_accessory_cost = []
state_vehicle_cost = []
for _, row in states_census_df.iterrows():
    state_purchase_row = states_purchases_df.loc[states_purchases_df['State'] == row['State']]
    state_weapon_cost.append(float(state_purchase_row['Weapons Purchase Amount']))
    state_accessory_cost.append(float(state_purchase_row['Tactical Equipment Purchase Amount']))
    state_vehicle_cost.append(float(state_purchase_row['Vehicle Purchase Amount']))

states_census_df['Weapons Purchase Amount'] = state_weapon_cost
states_census_df['Tactical Equipment Purchase Amount'] = state_accessory_cost
states_census_df['Vehicle Purchase Amount'] = state_vehicle_cost

In [114]:
writer = pd.ExcelWriter('State Census Purchase Combined.xlsx')
states_census_df.to_excel(writer,'Sheet1', index = False)
writer.save()